In [48]:
#!pip install -U ray

In [49]:
#!pip install -U "ray[tune]"  

In [50]:
# !pip install -U "ray[rllib]"  

In [51]:
# !pip install -U "ray[serve]"  

In [52]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
from sklearn.model_selection import KFold

import numpy as np
import os
import pandas as pd

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn



In [ ]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [53]:
"""
(Hyper)parameters of our convolutional neural network found by tuning with Ray Tune (see further on)

"""
input_size = 1 * 110 * 40
num_classes = 3
learning_rate = 0.07165149868611925
batch_size = 30
num_epochs = 10
torch.manual_seed(42)

## Complete Convolutional Neural Network
A first Convolutional Neural Network that takes on one input channel, where the PAC-matrix will be feeded as an input, and returns a probability array of total length equal to the amount of label classes. Each probability equals the chance of the label to correspond to the label class.

The network first consists of 3 Convolutional layers each of kernel size 3x3. Stride is set to its default value 1 and padding is initialised to 1 as well.
After each convolutional layer: first, a rectified linear unit activation function is applied and second a max pooling layer of kernel size 2x2 is used.

After the last convolutional layer, the output is flattened and fed into a first fully connected neural network layer where a tanh activation function is applied its the output. After this a random dropout with probability of dropping 0.25 is performed.
Finally a batch normalization is performed.

This procedure gets repeated after two more fully connected layers until the tensor after the third fully connected layer has a size equal to the amount of label classes.

As a final step this the softmax funtion is applied to this tensor, which transforms each element to its respective probability.

This CNN was designed purely for tuning the hyperparameters and getting general insights. The accuracies are not reported.


In [7]:
"""
Class inherits from nn.Module, a pytorch neural network of choice, must inherit from base class nn.Module.
"""
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        ) 
        
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        
        
        
        x = torch.randn(110, 40).view(-1, 1, 110, 40)
        self._to_linear = None
        
        self.dropout = nn.Dropout(0.25)
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 100) #flattening.
        self.fc2 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, num_classes) 
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.batchnorm2 = nn.BatchNorm1d(20)
        self.batchnorm3 = nn.BatchNorm1d(num_classes)     
    def convs(self, x):
        """
        Wrapper function for convolutional layers.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape 'self._to_linear'
        """   
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        if self._to_linear == None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            
        return x
    
    def forward(self, x):
        """
        Forward propagation function.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape (1, 100) serving as a feature vector.
        """
        x = self.convs(x) #execute convolutional layers      
        x = x.view(-1, self._to_linear) #flatten
        x = F.tanh(self.fc1(x))#pass through first fc layer
        x = self.dropout(x)
        x = self.batchnorm1(x)
        x = F.tanh(self.fc2(x))#pass through first fc layer
        x = self.dropout(x)
        x = self.batchnorm2(x)
        x = self.fc3(x) #final fc layer
        x = self.dropout(x)
        x = self.batchnorm3(x)
        return F.softmax(x, dim=1)

## Convolutional Neural Network for Feature Extraction
In order to use the PAC-matrix as a feature for a Multi-Layer Perceptron later on, the previous Convolutional Neural Network is slightly adapted to not return the tensor with label probabilities, but a tensor of shape (1, 100).

The network first consists of 3 Convolutional layers each of exactly the same parameters as the previous network.

After the last convolutional layer, the output is flattened and fed into a first fully connected neural network layer where a tanh activation function is applied its the output. After this a random dropout with probability of dropping 0.25 is performed.
Finally a batch normalization is performed.

After this the resulting tensor gets returned as a feature vector, which is fed along with other features to the final MLP.

In [54]:
"""
Class inherits from nn.Module, a pytorch neural network of choice, must inherit from base class nn.Module.
"""
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        ) 
        
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        
        
        
        x = torch.randn(110, 40).view(-1, 1, 110, 40)
        self._to_linear = None
        
        self.dropout = nn.Dropout(0.25)
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 100) #flattening.
        self.batchnorm1 = nn.BatchNorm1d(100)
        
    def convs(self, x):
        """
        Wrapper function for convolutional layers.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape ?
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        if self._to_linear == None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        """
        Forward propagation function.
        
        self: self
        x: tensor of shape (num_channels, PAC_y_dim, PAC_x_dim)
            where PAC_y_dim and PAC_x_dim are the dimensions of the PAC-matrix
        
        returns: tensor of shape (1, 100) serving as a feature vector.
        """
        x = self.convs(x) #execute convolutional layers
        x = x.view(-1, self._to_linear) #flatten
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.batchnorm1(x)
        return x

In [55]:
def reset_weights(m):
  """
  Try resetting model weights to avoid
  weight leakage.
    
  m: a PyTorch model
    
  return: None
  """
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [56]:
"""
Set device cuda for GPU if it's available otherwise run on the CPU
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load labels

In [57]:
"""
Load labels from csv file
"""
labels = np.genfromtxt('../Data/Labels/avg_smooth.csv', delimiter=',')

## Quantify raw label values
After reading the raw label values, they're quantified to 3 discrete levels: 0, 1 or 2.

0: 'No fear apparent in rodents brain activity'

1: 'A small amount of fear could be noticed, but too noisey to be sure.'

2: 'Rodent is fearful'

In [58]:
df_labels = pd.DataFrame(labels)
df_labels = pd.cut(df_labels[0],bins=[0, 0.012, 0.017, 0.03],labels=[0,1,2])
labels = df_labels.to_numpy()


print('\nAmount of non-fearful rodents:' + str(len([i for i in list(labels) if i==0])))
print('Amount of little fearful rodents:' + str(len([i for i in list(labels) if i==1])))
print('Amount of fearful rodents:' + str(len([i for i in list(labels) if i==2])))


Amount of non-fearful rodents:153
Amount of little fearful rodents:251
Amount of fearful rodents:196


## Load all PAC-matrices from .npy-files

In [60]:
"""
Load .npy files into one big array
"""
data = []
dir = '../Data/Feature arrays/'
for filename in os.listdir(dir):
    sample = np.load(dir+filename)
    data.append(sample[0]) #select only PAC
data = np.array(data[:-1])
data = np.expand_dims(data,1)
print('Feature shapes: ' + str(data.shape), '\nLabel vector shape: ' + str(labels.shape))

Feature shapes: (600, 1, 110, 40) 
Label vector shape: (600,)


In [61]:
"""
Transform data to pytorch tensors
"""
tensor_x = torch.Tensor(data)
tensor_y = torch.Tensor(labels)
tensor_y = tensor_y.type(torch.LongTensor)

In [19]:
"""
Create pytorch dataset and dataloader
"""
dataset = TensorDataset(tensor_x,tensor_y)

"""
Split into train and test sets
"""
test_size = int(0.3*len(dataset))
train_size = len(dataset) - test_size

train_data,test_data = random_split(dataset,[train_size,test_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Test Data : {len(test_data)}")

train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)

Length of Train Data : 420
Length of Test Data : 180


In [62]:
"""
Initialize network
""" 
model = CNN().to(device)

"""
Use CrossEntropyLoss and Adam optimizer
""" 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Define accuracy function

In [63]:
"""
 Check accuracy on training & test to see how good the model is
 
 loader: a pytorch dataloader
 model: a pytorch model
 
 returns: model accuracy as a numerical value
"""
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

## Apply crossvalidation
Apply 5-fold crossvalidation with CrossEntropy loss and Adam optimizer.

In [64]:
"""
Define the 5-fold Cross Validator
""" 
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=False)

In [53]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')
    model.apply(reset_weights)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    trainloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=test_subsampler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for batch_idx, (input_, labels) in enumerate(tqdm(trainloader)):
            # Get data to cuda if possible
            input_ = input_.to(device=device)
            targets = labels.to(device=device)

            # forward propagation
            scores = model(input_)
            loss = criterion(scores, targets)

            # backpropagation
            optimizer.zero_grad()
            loss.backward()

            # adam step
            optimizer.step()
    print('Train Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(trainloader, model)))
    print('Test Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(testloader, model)))

Fold 0


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 68.09it/s]


Train Accuracy for fold 0: 73 %
Test Accuracy for fold 0: 61 %
Fold 1


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 77.30it/s]


Train Accuracy for fold 1: 66 %
Test Accuracy for fold 1: 84 %
Fold 2


...

Train Accuracy for fold 2: 68 %
Test Accuracy for fold 2: 75 %
Fold 3


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 68.47it/s]


Train Accuracy for fold 3: 73 %
Test Accuracy for fold 3: 58 %
Fold 4


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 60.61it/s]


Train Accuracy for fold 4: 70 %
Test Accuracy for fold 4: 71 %


## Save extracted feature vector to .npy file to feed it into MLP

In [43]:
features = model.forward(tensor_x.to(device=device))
features = features.cpu().detach().numpy()
features = np.mean(features, axis=0)
np.save('../Data/PAC_afterCNN.npy', features)

## Hyperparameter Tuning with Ray Tune
As noted before some parameters where acquired in their optimal form by performing hyperparameter tuning.  
The library used for this was Ray Tune.  

Following hyperparameters where tuned:
- number of epochs
- learning rate
- batch size

with following optimal values:  
- number of epochs = 10
- learning_rate = 0.07165149868611925
- batch_size = 30


In [42]:
config = {
    "num_epochs": tune.choice([2, 5, 10]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([5, 10, 50, 100])
}
max_num_epochs = 11

In [44]:
"""
Ray Tune requires one method to contain all necessary network definitions and methods.

config: dictionary with configuration hyperparameters
checkpoint_dir: not used, but could be used to load earlier configuration

returns: None
"""
def train(config, checkpoint_dir=None):
    model = CNN().to(device)
    model.apply(reset_weights)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    batch_size = config["batch_size"]
    train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    for epoch in range(config["num_epochs"]):
        running_loss = 0
        epoch_steps = 0
        for batch_idx, (data, labels) in enumerate(tqdm(train_loader)):
            current_loss = 0
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = labels.to(device=device)
            # Get to correct shape
            ###data = data.reshape(data.shape[0], -1)
            ###data = data.reshape(-1, )
            ###data = data.reshape(-1, 3, 110, 40)
            #print(data.shape)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()

            #print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if batch_idx%500 == 499:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
                
                
        test_loss = 0
        test_steps = 0
        total = 0
        correct = 0
        for batch_idx, (data, labels) in enumerate(tqdm(test_loader)):
            with torch.no_grad():
                data = data.to(device=device)
                targets = labels.to(device=device)
                scores = model(data)
                _, predictions = scores.max(1)
                total += predictions.size(0)
                correct += (predictions == targets).sum().item()
                loss = criterion(scores, targets)
                test_loss += loss.cpu().numpy()
                test_steps+=1
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=(test_loss/test_steps), accuracy=correct/total)
    print("Finished")

            
    

In [55]:
"""
num_samples: amount of hyperparameter value combinations to check
gpus_per_trial: amount of gpus available

return: None
"""
def main(num_samples = 10, gpus_per_trial=1):
    config = {
    "num_epochs": tune.choice([4, 5, 10]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([10, 30, 50, 100])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuray", "train_iteration"])
    result = tune.run(
            partial(train),
            resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
            config=config,
            num_samples = num_samples,
            scheduler=scheduler,
            progress_reporter=reporter)
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    #best_trained_model = model(best_trial)

In [58]:
main(num_samples=10, gpus_per_trial=1)

2021-12-18 16:19:02,462	WARNING experiment.py:255 -- No name detected on trainable. Using DEFAULT.
2021-12-18 16:19:02,463	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.
2021-12-18 16:19:06,390	WARNING worker.py:1245 -- Warning: The actor ImplicitFunc is very large (10 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


== Status ==
Current time: 2021-12-18 16:19:06 (running for 00:00:03.78)
Memory usage on this node: 15.4/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/20.19 GiB heap, 0.0/10.09 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-19-02
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_d5176_00000 | RUNNING  | 127.0.0.1:26304 |           10 |     0.000667685 |            5 |
| DEFAULT_d5176_00001 | PENDING  |                 |           10 |     0.00173855  |            5 |
| DEFAULT_d5176_00002 | PENDING  |                 |     

  0%|          | 0/42 [00:00<?, ?it/s]


== Status ==
Current time: 2021-12-18 16:19:11 (running for 00:00:09.00)
Memory usage on this node: 16.7/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/20.19 GiB heap, 0.0/10.09 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-19-02
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_d5176_00000 | RUNNING  | 127.0.0.1:26304 |           10 |     0.000667685 |            5 |
| DEFAULT_d5176_00001 | PENDING  |                 |           10 |     0.00173855  |            5 |
| DEFAULT_d5176_00002 | PENDING  |                 |     

 pid=26304) C:\Users\Tomas\anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
 pid=26304)   warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
  0%|          | 0/18 [00:00<?, ?it/s]


...

Result for DEFAULT_d5176_00009:
  accuracy: 0.4166666666666667
  date: 2021-12-18_16-24-05
  done: false
  experiment_id: 0c1cf5a6637840528d0ddfac04451852
  hostname: WSTOMAS21
  iterations_since_restore: 1
  loss: 1.0861102938652039
  node_ip: 127.0.0.1
  pid: 20408
  should_checkpoint: true
  time_since_restore: 9.132593154907227
  time_this_iter_s: 9.132593154907227
  time_total_s: 9.132593154907227
  timestamp: 1639841045
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d5176_00009
  


  0%|          | 0/9 [00:00<?, ?it/s]


== Status ==
Current time: 2021-12-18 16:24:07 (running for 00:05:05.04)
Memory usage on this node: 17.3/47.6 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 8.000: -0.9079552988211314 | Iter 4.000: -0.9210878908634186 | Iter 2.000: -0.9694160421689352 | Iter 1.000: -1.1104625629054174
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/20.19 GiB heap, 0.0/10.09 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-19-02
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |     loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+----------|
| DEFAULT_d5176_00009 | RUNNING    | 127.0.0.1:20408 |           50 |     0.0215958   |            4 | 1.08611  |
| DEFAULT_d5176_00000 | TERMINATED | 127.0.

100%|██████████| 4/4 [00:03<00:00,  1.20it/s]
2021-12-18 16:24:12,535	INFO tune.py:626 -- Total run time: 310.07 seconds (309.67 seconds for the tuning loop).


Result for DEFAULT_d5176_00009:
  accuracy: 0.5166666666666667
  date: 2021-12-18_16-24-12
  done: true
  experiment_id: 0c1cf5a6637840528d0ddfac04451852
  hostname: WSTOMAS21
  iterations_since_restore: 2
  loss: 1.0201688557863235
  node_ip: 127.0.0.1
  pid: 20408
  should_checkpoint: true
  time_since_restore: 15.806646347045898
  time_this_iter_s: 6.674053192138672
  time_total_s: 15.806646347045898
  timestamp: 1639841052
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: d5176_00009
  
== Status ==
Current time: 2021-12-18 16:24:12 (running for 00:05:09.68)
Memory usage on this node: 17.2/47.6 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: -0.9079552988211314 | Iter 4.000: -0.9210878908634186 | Iter 2.000: -0.9947924489776294 | Iter 1.000: -1.1104625629054174
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/20.19 GiB heap, 0.0/10.09 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-19-02
Number of trials: 10/10 (10 TERMINATED)
+-